In [ ]:
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append('../../')
from feature import selection as sel

# Mutual information

## Time domain

### Unnormalized vs. Normalized features
- Result found: Mutual information is independent of scaling

In [ ]:
features = sel.load_time_domain_features(['ax', 'ay', 'az', 'bx', 'by', 'bz'])
fscore = sel.calc_mutual_information(features, sel.TD_COLUMNS, summary=True)

features_normalized = sel.normalize_features(features)
fscore_norm = sel.calc_mutual_information(features_normalized, sel.TD_COLUMNS, summary=True)

fig, ax = plt.subplots(1, 2, figsize=(20, 5))
fscore.plot.bar(figsize=(10, 4), grid=True, xlabel='Feature', ylabel='Mutual information', legend=False, title='Unnormalized', ax=ax[0])
fscore_norm.plot.bar(figsize=(10, 4), grid=True, xlabel='Feature', ylabel='Mutual information', legend=False, title='Normalized', ax=ax[1])
plt.show()

### MI between feature on axis and target fault state

In [ ]:
features = sel.load_time_domain_features(['ax', 'ay', 'az', 'bx', 'by', 'bz'])
df = sel.calc_mutual_information(features, sel.TD_COLUMNS, summary=False)
fig, ax = plt.subplots(figsize=(10, 5)) 
sb.heatmap(df, annot=True, ax=ax, cmap="Greens", fmt=".3f")
plt.show()
df

## Frequency domain

### F score of all windows in all axis to multiclass fault

In [ ]:
EXCLUDE_COLS = ['inharmonicity', 'hdev', 'negentropy']
columns = list(set(sel.FD_COLUMNS) - set(EXCLUDE_COLS))

features = sel.load_frequency_domain_features(['ax', 'ay', 'az', 'bx', 'by', 'bz'])
features = features.drop(columns=EXCLUDE_COLS)

fscore = sel.calc_mutual_information(features, columns, summary=True)
fscore.plot.bar(figsize=(5, 4), grid=True, xlabel='Feature', ylabel='Mutual information', legend=False, title='F score in Frequency domain')
plt.show()

### MI to some faults

In [ ]:
features_chosen = features[features['fault'].isin([
    'normal', 'imbalance', 'vertical-misalignment', 'horizontal-misalignment' 
])]
fscore = sel.calc_mutual_information(features_chosen, columns, summary=True)
fscore.plot.bar(figsize=(5, 4), grid=True, xlabel='Feature', ylabel='F statistic', legend=False, title='F score in Frequency domain')
plt.show()

### MI in all axis to multiclass fault (per each window size)

In [ ]:
df = sel.calc_score_in_fft_windows(features, columns, lambda f, c: sel.calc_mutual_information(f, c, summary=True))
sel.plot_fscore_part(df, 'window')

#### All faults: best features by ranking over all windows (non-weighted vs. weighted by score)
- less is better

In [ ]:
sel.plot_rank(df, 'window')

### MI in all axis to multiclass fault (per each window size) and chosen faults

In [ ]:
df_chosen = sel.calc_score_in_fft_windows(features_chosen, columns, lambda f, c: sel.calc_mutual_information(f, c, summary=True))
sel.plot_fscore_in_fft_win(df_chosen)

#### All faults: best features by ranking over all windows (non-weighted vs. weighted by score)

In [ ]:
sel.plot_rank(df_chosen, 'window')

## Wavelet packet transform

In [ ]:
features = sel.load_wavelet_domain_features(['ax', 'ay', 'az', 'bx', 'by', 'bz'])
df = sel.calc_score_in_wpd_features(features, lambda f, c: sel.calc_mutual_information(f, c, summary=True))
sel.plot_fscore_part(df, 'metric', n=30)

#### WPD features in one layer

In [ ]:
level = 3
df = sel.calc_score_in_wpd_features(features, lambda f, c: sel.calc_mutual_information(f, c, summary=True))
layer = df[df.index.str.startswith(f'L{level}')]
sel.plot_fscore_part(layer, 'metric')

In [ ]:
sel.plot_rank(layer, 'metric')

In [ ]:
level = 4
df = sel.calc_score_in_wpd_features(features, lambda f, c: sel.calc_mutual_information(f, c, summary=True))
layer = df[df.index.str.startswith(f'L{level}')]
sel.plot_fscore_part(layer, 'metric')

### Mutual information between feature in axis and various faults (predicted variable)

In [ ]:
def mi_among_fault_and_axis(features, cols):
    fig, ax = plt.subplots(5, 1, figsize=(8, 20))

    WINDOW_SIZES = (2**8, 2**10, 2**12, 2**14, 2**16)
    for i, win in enumerate(WINDOW_SIZES):
        x = features[
            (features['fft_window_length'] == win) &
            (features['axis'].isin(cols))
        ].dropna()
        o = ax.flatten()[i]
        mi = sel.calc_mutual_information(x, sel.FD_COLUMNS, summary=False)
        sb.heatmap(mi, annot=True, ax=o, cmap="Greens")
        o.set_title(f'FFT: {win}')

AXIS = ['ax', 'ay', 'az', 'bx', 'by', 'bz']
features = pd.read_csv(sel.FREQ_FEATURES_PATH)
features['fault'] = features['fault'].astype('category')
features['fft_window_length'] = features['fft_window_length'].astype('category')

mi_among_fault_and_axis(features, AXIS)
plt.show()

## MI in Wavelets

In [ ]:
features = pd.read_csv(sel.WPD_FEATURES_PATH)

WPD_AXIS = 'ax'
# More axis at once significantly reduces MI
features = features[features['axis'] == WPD_AXIS]                 # One axis
features['fault'] = features['fault'].astype('category')
#features = features[features['axis'].isin(['ax', 'ay', 'az'])]  # One measuremnt position

columns = [col for col in features.columns 
           if col not in ('fault', 'severity', 'seq', 'rpm', 'axis', 'feature')]
features.head()

In [ ]:
features_energy = features[features['feature'] == 'energy']
print(len(features_energy))

mi = sel.calc_mutual_information(features_energy, columns, summary=True)
mi.iloc[:30].plot.bar(figsize=(20, 4), grid=True, ylabel='MI', title='WPD Energy')
plt.show()

In [ ]:
def plot_wpd_energy_ratio_per_level(features, wpd_axis):
    features = features[features['axis'].isin(wpd_axis)]  
    features_energy_ratio = features[features['feature'] == 'energy_ratio']
    # print(len(features_energy_ratio))
    
    fig, ax = plt.subplots(6, 1, figsize=(15, 20))
    
    for level in range(1, 7):
        cols = np.array(columns)
        cols = cols[np.char.startswith(cols, f'L{level}')]
        mi = sel.calc_mutual_information(features_energy_ratio, cols, summary=True)
        
        o = ax.flatten()[level-1]
        o.bar(mi.index, mi.values.T[0])
        o.grid(True)
        o.set_xlabel('Feature')
        o.set_ylabel('MI')
        
        # Rotate x labels by 45 deg
        o.set_xticks(o.get_xticks())
        o.set_xticklabels(o.get_xticklabels(), rotation=45, ha='right')

    fig.suptitle(f'WPD energy ratio: Axis "{wpd_axis}"', fontsize=16, y=0.9)
    plt.show()

In [ ]:
plot_wpd_energy_ratio_per_level(features, ['ax'])

In [ ]:
plot_wpd_energy_ratio_per_level(features, ['ax', 'ay', 'az'])

In [ ]:
features_entropy = features[features['feature'] == 'negentropy']
print(len(features_entropy))

mi = sel.calc_mutual_information(features_entropy, columns, summary=True)
mi.iloc[:30].plot.bar(figsize=(20, 4), grid=True, ylabel='MI', title='WPD Negentropy')
plt.show()

In [ ]:
features_kurtosis = features[features['feature'] == 'kurtosis']
print(len(features_kurtosis))

mi = sel.calc_mutual_information(features_entropy, columns, summary=True)
mi.iloc[:30].plot.bar(figsize=(20, 4), grid=True, ylabel='MI', title='WPD Kurtosis')
plt.show()

In [ ]:
def level_to_frequency_bands(level, fs):
    bin_count = 2 ** level
    bin_width = (fs / 2) / bin_count
    for bin in range(bin_count):
        a = bin * bin_width
        b = a + bin_width
        print(f'L{level}_{bin} = [{a}; {b}] Hz')

level_to_frequency_bands(level=4, fs=50000)